# Baseline

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Загрузка данных

In [ ]:

train_df = pd.read_parquet('drive/MyDrive/Colab Notebooks/train.parquet')
test_df = pd.read_parquet('drive/MyDrive/Colab Notebooks/test.parquet')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df.head()

,id,rko_start_months,max_end_fact_fin_deals,max_end_plan_non_fin_deals,max_start_fin_deals,max_start_non_fin_deals,min_end_fact_fin_deals,min_end_plan_non_fin_deals,min_start_fin_deals,min_start_non_fin_deals,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,target_1,target_2,total_target
0,0,-1.279132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.352516,-0.081310,0.210189,1.140808,0.647477,0.810887,1.909568,0,0,0
1,1,-1.142591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.188287,-0.650616,-0.316563,-1.136536,-0.127326,-0.147319,-0.517805,0,0,0
2,2,1.812270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.323487,0.632731,2.541558,3.017482,0.033394,0.019972,0.578428,0,0,0
3,3,-0.479407,NaN,0.356677,NaN,-0.332867,NaN,-0.704164,NaN,0.905748,...,-0.323487,-0.003549,-0.219016,0.228624,-0.428185,-0.220932,-0.596108,0,0,0
4,4,-1.503690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.323487,0.182726,-0.287299,-0.528921,-0.563335,-0.021506,-0.361201,0,0,0


In [ ]:
test_df.head()

,id,rko_start_months,max_end_fact_fin_deals,max_end_plan_non_fin_deals,max_start_fin_deals,max_start_non_fin_deals,min_end_fact_fin_deals,min_end_plan_non_fin_deals,min_start_fin_deals,min_start_non_fin_deals,...,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m
0,300000,-1.076009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.514610,-0.045603,-0.131653,-0.323487,-0.169940,-0.316563,-0.639934,-0.171600,-0.237763,-0.596108
1,300001,-0.335070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.514610,-0.045603,-0.131653,-0.323487,-0.169940,-0.316563,-0.639934,-0.171580,-0.222361,-0.517805
2,300002,0.920834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.463706,-0.045603,-0.131653,-0.323487,-0.169940,-0.316563,-0.639934,0.020943,-0.160754,-0.204596
3,300003,-0.772897,-0.740714,NaN,-0.635375,NaN,-0.595302,NaN,-0.738424,NaN,...,-0.514610,-0.045603,-0.131653,-0.323487,-0.137993,-0.277545,-0.381706,-0.171600,-0.237763,-0.596108
4,300004,-0.665342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.209184,-0.045603,-0.131653,-0.323487,-0.139926,-0.277545,-0.381706,-0.158918,-0.222361,-0.517805


Для базовой модели отбросим отдельные таргеты и будем использовать только total_target.

Преобразуем тип категориальных признаков

## Разбиение на train, validation

## Обучение базовой модели

В качестве базовой модели возьмем LGBMClassifier

In [21]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
train_df = pd.read_parquet('drive/MyDrive/Colab Notebooks/train.parquet')
test_df = pd.read_parquet('drive/MyDrive/Colab Notebooks/test.parquet')

# Удаление отдельных таргетов и использование только total_target
train_df.drop(["id", "target_1", "target_2"], axis=1, inplace=True)

# Преобразование категориальных признаков
cat_cols = [
    'channel_code', 'city', 'city_type',
    'index_city_code', 'ogrn_month', 'ogrn_year',
    'branch_code', 'okved', 'segment'
]

# Используем Label Encoding для категориальных признаков


train_df[cat_cols] = train_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

# Создание новых признаков (инженерия признаков)
train_df['max_min_end_fact_fin_deals'] = train_df['max_end_fact_fin_deals'] + train_df['min_end_fact_fin_deals']
test_df['max_min_end_fact_fin_deals'] = test_df['max_end_fact_fin_deals'] + test_df['min_end_fact_fin_deals']

# Разбиение на train и validation с использованием StratifiedKFold
X = train_df.drop("total_target", axis=1)
y = train_df.total_target

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)



model = LGBMClassifier(verbosity=-1, random_state=42, n_estimators=1000, max_depth=8, learning_rate=0.05)
model.fit(x_train, y_train)

y_pred = model.predict_proba(x_val)[:, 1]

roc_auc = roc_auc_score(y_val, y_pred)

print(roc_auc)

0.8668837730776122


Качество получилось довольно неплохим, но его еще можно улучшить

## Выгрузка результатов

In [22]:
test_score = model.predict_proba(test_df.drop("id", axis=1))[:, 1]

In [23]:
sample_submission_df = pd.read_csv("drive/MyDrive/Colab Notebooks/sample_submission.csv")

In [ ]:
sample_submission_df.head()

,id,score
0,300000,0.5
1,300001,0.5
2,300002,0.5
3,300003,0.5
4,300004,0.5


In [24]:
sample_submission_df["score"] = test_score

In [25]:
sample_submission_df.head()

,id,score
0,300000,0.422596
1,300001,0.408913
2,300002,0.015620
3,300003,0.005372
4,300004,0.002394


In [26]:
sample_submission_df.to_csv("my_submission.csv", index=False)